Welcome to week 3 of the Noisebridge Python Class ([Noisebridge Wiki](https://www.noisebridge.net/wiki/PyClass) | [Github](https://github.com/audiodude/PythonClass))

This week, we will learn about **exceptions**. Let's face it, the world is messy, and sometimes things will go wrong. We are all familiar with the basic concept of a "computer error". In Python, we use exceptions to more precisely define, reason about, and handle errors.

We will learn about:

* Exception types, including built-in Exceptions
* What happens when an exception occurs
* Catching exceptions
* Ignoring exceptions (!)
* Defining our own exceptions

That's what we'll learn, no exceptions, unless I've made an error....

---

Python has the concept of `Exceptions`, which interrupt program flow and represent a condition that a running program either didn't expect, or can't recover from (an error). Exceptions cover everything from incorrect program syntax, and performing operations on data types that don't support them (like, say, using slice notation on an integer), to attempting to read data from a list or dict at an index that doesn't exist or trying to read in a file that is missing.

All exceptions in Python are **subclasses** of the `BaseException` class. `BaseException` has a further subclass tree starting with `Exception`. The former includes things like `SystemExit` which generally shouldn't be caught, while the latter contains things like `KeyError` which we sometimes want to catch.

What does it mean to "catch" an Exception anyways? Let's look at an example.

In [ ]:
number_strings = ('1', '2', 'x', '4')
my_numbers = [int(n) for n in number_strings]
print('Done converting numbers')

This code raises an exception, because the string 'x' cannot be turned into an integer. When the exception is raised, all program execution stops immediately and the interpreter looks for exception handling code. If it doesn't find any, the exception is raised to the "top level" of the program, where it will subsequently halt the program completely (the program "crashes").

It is very obvious in this example that the error would be thrown, but what if the 'x' was hidden somewhere in a dirty data file? Let's see how we can catch the exception.

In [ ]:
number_strings = ('1', '2', 'x', '4', 'foo')
try:
    my_numbers = [int(n) for n in number_strings]
except ValueError as e:
    print('Could not convert some values: %s' % e)

print('Done converting numbers')
    
# What does the my_numbers list contain?  
# print(my_numbers)

Did you notice something about `my_numbers`? It never actually gets populated. Even though we "caught" the exception, Python stopped processing the list comprehension, and `my_numbers` never got assigned a value. A possible better way to write this code is without a list comprehension (gasp!).

In [ ]:
number_strings = ('1', '2', 'x', '4', 'foo')
my_numbers = []
rejects = []
for string in number_strings:
    try:
        my_numbers.append(int(string))
    except ValueError:
        rejects.append(string)
        
print(f'Converted {my_numbers}, but found {len(rejects)} rejects: {rejects}')

Generally, you should only catch exceptions if you intend to do something meaningful when they occur, even if that just means logging them. It is possible to 'swallow' exceptions:

In [ ]:
my_numbers = [42, 34, 100]
try:
    x = my_numbers[5]
except IndexError:
    pass

Here we introduce `pass`, a special Python keyword which means "do nothing" or "no-op". It's not possible in Python, mostly due to whitespace processing, to have an "empty" block. So this doesn't work:

In [ ]:
def my_func():

You need to use `pass`:

In [ ]:
def my_func():
    pass

This is most useful when defining classes or functions that you don't know what they will do yet, or you're not ready to write their implementations.

Back to exceptions. We can catch multiple exceptions in the same except block:

In [ ]:
number_strings = ('1', '2', '4')
try:
    my_numbers = [int(n) for n in number_strings]
    my_numbers[100]
except (ValueError, IndexError) as e:
    print('Could not convert some values: %s' % e)

We can also have separate blocks for each exception or groups of exceptions:

In [ ]:
number_strings = ('1', '2', '4')
try:
    my_numbers = [int(n) for n in number_strings]
    my_numbers[100]
except ValueError as e:
    print('Could not convert some values: %s' % e)
except IndexError:
    print('One of the indexes was not valid')

Note in the above: if we don't need to do anything with the exception message or stacktrace, we can omit the `as _` clause.

We can also re-raise an exception if we've done as much as we can do to remedy the situation, but we want it to "bubble up" another level.

In [ ]:
def parse_string(s):
    try:
        return int(s)
    except ValueError:
        print('{} is not an int'.format(s))
        raise
        
def parse_all_strings(strings):
    for string in strings:
        try:
            parse_string(string)
        except IndexError:
            # This error never happens
            print('Somehow an index error occurred')
        # Since we don't catch the ValueError, it is passed
        # all the way back up to main()
        
def main():
    my_strings = ['42', '34', 'x', '100']
    try:
        numbers = parse_all_strings(my_strings)
        print(f'Numbers are: {numbers}')
    except ValueError as e:
        # We can catch this exception here because it is
        # raised from parse_all_strings()
        print('Not all strings could be parsed: %s' % e)
        
        
try:
    main()
# Careful with this, it will catch any errors in your program logic,
# which will make it hard to debug while writing your program.
except Exception as e:
    # The exception wasn't re-raised from main() so this doesn't
    # get called
    print('Program had errors!')
finally:
    print('Program done')

In the example above, the exception occurs in `parse_string`, which logs it and calls `raise` to re-raise it. It then passes straight through `parse_all_strings`, which doesn't define an exception handler for `ValueError` and is re-caught in `main()`. Notice that 'Numbers are: []' never gets printed, because the exception happens before that line.

The other interesting new construct here is the `finally` clause. Any code here, after a `try` (with or without an `except`) will get executed no matter what, whether there are exceptions or not, or what kind of exceptions occur. This is most commonly used for "clean up" code, like if you opened a file (and didn't use a context manager, with...as) and want to make sure it gets closed even if there is an exception

Finally (see what I did there?), we can define our own exceptions that inherit from `Exception`:

In [ ]:
class PlaygroundException(Exception):
    pass

class NameNotGivenError(PlaygroundException):
    pass

class LocationMissingError(PlaygroundException):
    pass

In [ ]:
students = (
    ('Alice', 'slide'),
    ('Bob', 'sandbox'),
    ('Claire', None),
    (None, 'slide'),
)

def rollcall():
    for student in students:
        if student[0] is None:
            raise NameNotGivenError('Missing name')
        if student[1] is None:
            raise LocationMissingError('No location')
        print(f'{student[0]} is at the {student[1]}')

try:
    rollcall()
except NameNotGivenError as e:
    print('Error: %s' % e)
except PlaygroundException as e:
    print('Playground error: %s' % e)

The resolution of exceptions follows the **class hierarchy** of the exceptions themselves. Here, we don't explicitly handle the `LocationMissingError`, but it is handled inside the block for `PlaygroundException` because it **is a** `PlaygroundException`. If there are multiple handlers in a try/except block that could handle an exception, they are tested in order.